In [23]:
# Работа с данными
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import datasets
from IPython.display import display

# Визуализация данных
import matplotlib.pyplot as plt
import seaborn as sns

# алгоритмы
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

# Ансамбль
from sklearn.ensemble import VotingClassifier

# Подбор гиперпараметров
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

# Метрики
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error

# Загрузка файла
from google.colab import files
import csv

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

### **1 Шаг**
#### Загрузка dataset-а и изучение полученных данных

In [7]:
# Загрузка файла
data = files.upload()

# Извлечение имени файла
filename = next(iter(data))

# Чтение CSV-файла в DataFrame с учетом возможных проблем
try:
    games_df_start = pd.read_csv(filename, sep=',', encoding='utf-8', on_bad_lines='skip', quoting=csv.QUOTE_NONE)
    games_df_start.info()
except Exception as e:
    print(f"Произошла ошибка: {e}")

Saving hackatonmoscow.csv to hackatonmoscow (2).csv


<ipython-input-7-3aaa6f9dde44>:9: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  games_df_start = pd.read_csv(filename, sep=',', encoding='utf-8', on_bad_lines='skip', quoting=csv.QUOTE_NONE)


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1005215 entries, ('"596118586', 'Меджид', 'Кузарович', 'Евдунов') to (';https://media.komus.ru/medias/sys_master/root/hb8/h60/10148101816350.jpg;https://media.komus.ru/medias/sys_master/root/hf6/h8b/10148102209566.jpg;https://media.komus.ru/medias/sys_master/root/hff/h41/10148102602782.jpg;https://media.komus.ru/medias/sys_master/root/h84/h50/10148102995998.jpg;https://media.komus.ru/medias/sys_master/root/hfe/h7b/10148103389214.jpg;"https://media.komus.ru/medias/sys_master/root/h0b/hb6/10148103', 'VAT_20', '""0', '072""')
Data columns (total 47 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   "client_id                403 non-null    object
 1   ""client_first_name""     250 non-null    object
 2   ""client_middle_name""    246 non-null    object
 3   ""client_last_name""      251 non-null    object
 4   ""client_fio_full""       404 non-null    object
 5   ""cli

In [8]:
# Проверка на пропущенные знаки в DataFrame
games_df_start.isnull().sum()

,0
"""client_id",1004812
"""""client_first_name""""",1004965
"""""client_middle_name""""",1004969
"""""client_last_name""""",1004964
"""""client_fio_full""""",1004811
"""""client_bday""""",1004812
"""""client_bplace""""",1004819
"""""client_cityzen""""",1004824
"""""client_resident_cd""""",1004975
"""""client_gender""""",1004973


#### Вывод по полученным данным:
Из анализа данных следует, что в датасете отсутствует информация о 2 названиях игр, 269 годах выпуска, 2 жанрах, 8578 оценках критиков, 6701 оценках пользователей и 6766 оценках от организации ESRB. Возможные причины пропусков могут быть связаны с ошибками при сборе данных, недоступностью информации или неправильным сохранением данных в источнике.

Наиболее важными признаками для обработки пропусков являются оценки критиков, пользователей и ESRB.

### **2 Шаг**
#### Подготовка данных

In [9]:
# Заменим названия столбцов, приведем к нижнему регистру
games_df_start.columns = games_df_start.columns.str.lower()

In [10]:
games_df_start.head(20)

,,,,"""client_id","""""client_first_name""""","""""client_middle_name""""","""""client_last_name""""","""""client_fio_full""""","""""client_bday""""","""""client_bplace""""","""""client_cityzen""""","""""client_resident_cd""""","""""client_gender""""",...,"""""fin_loan_value""""","""""fin_loan_debt""""","""""fin_loan_percent""""","""""fin_loan_begin_dt""""","""""fin_loan_end_dt""""","""""stream_favorite_show""""","""""stream_duration""""","""""create_date""""","""""update_date""""","""""source_cd"""""";;;;;;;;"
"""596118586",Меджид,Кузарович,Евдунов,Евдунов Меджид Кузарович,"""""1992-08-23""""",ДЕР. КИРГИНЦЕВО АРОМАШЕВСКОГО Р-НА ТЮМЕНСКОЙ ОБЛ.,RUS,Д,М,Д,Н,2,Д,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-16 05:10:57.904,2020-07-16 05:10:57.904,"Bank"";;;;;;;;"
"""747208737",Мансыр,Хирсанович,Повод,Повод Мансыр Хирсанович,"""""1987-03-15""""","""""""""","""""""""",,М,,"""""""""",1,,...,NaN,Star Wars: A New Hope,1187.0,2020-11-13 21:33:19.620,2020-11-13 21:33:19.620,"Stream"";;;;;;;;",NaN,NaN,NaN,NaN
"""625104053",НУРДИНБЕК,УВАЙСОВИЧ,БУРНАС,БУРНАС НУРДИНБЕК УВАЙСОВИЧ,"""""1961-04-05""""",СЕЛО ДИОРОВА ТЯЧЕВСКОГО РАЙОНА ЗАКАРПАТСКОЙ ОБ...,RUS,,М,,"""""""""",NaN,,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-09-06 20:41:55.362,2022-09-06 20:41:55.362,"Telco"";;;;;;;;",NaN
"""624196811",Афаят,Гордиевна,Бондажевская,Бондажевская Афаят Гордиевна,"""""1976-06-01""""","""""""""","""""""""",,Ж,,"""""""""",0,,...,NaN,Остаться в живых,35.0,2021-08-17 19:00:45.024,2021-08-17 19:00:45.024,"Stream"";;;;;;;;",NaN,NaN,NaN,NaN
"""177500487",Сахия,Фидаевна,Разуева,Разуева Сахия Фидаевна,"""""1961-07-31""""","""""""""","""""""""",,Ж,,"""""""""",2,,...,NaN,Начало,834.0,2022-11-05 05:37:58.131,2022-12-04 00:00:09.068,"Stream"";;;;;;;;",NaN,NaN,NaN,NaN
"""81705848",Аржана,Наилович,Чередник,Чередник Аржана Наилович,"""""1933-07-06""""","""""""""","""""""""",,Ж,,"""""""""",1,,...,NaN,Мстители: Финал,1334.0,2022-11-11 11:49:24.679,2022-11-11 11:49:24.679,"Stream"";;;;;;;;",NaN,NaN,NaN,NaN
"""445441055",Аимкан,Агургеновна,Богач,Богач Аимкан Агургеновна,"""""1943-11-19""""","""""""""","""""""""",,Ж,,"""""""""",1,,...,NaN,Шерлок,845.0,2024-07-26 17:11:39.934,2024-08-21 07:46:25.748,"Stream"";;;;;;;;",NaN,NaN,NaN,NaN
"""333808481",Арли,Янис,Помилов,Помилов Арли Янис,"""""1963-10-01""""","""""""""","""""""""",,М,,"""""""""",3,,...,NaN,Очень странные дела,831.0,2023-08-31 23:23:19.436,2024-03-19 21:49:48.631,"Stream"";;;;;;;;",NaN,NaN,NaN,NaN
"""610269013",Магарам,Тайевич,Лыло,Лыло Магарам Тайевич,"""""1926-07-26""""","""""""""","""""""""",,М,,"""""""""",1,,...,NaN,Во все тяжкие,1162.0,2021-07-23 06:06:36.147,2023-01-30 00:31:47.680,"Stream"";;;;;;;;",NaN,NaN,NaN,NaN
"""996443530",Тойдык,Чукановна,Лехно,Лехно Тойдык Чукановна,"""""1965-09-11""""","""""""""","""""""""",,Ж,,"""""""""",0,,...,NaN,Парк юрского периода,80.0,2021-02-08 00:14:20.275,2021-02-08 00:14:20.275,"Stream"";;;;;;;;",NaN,NaN,NaN,NaN


#### Обработка пропусков

###### Используем функцию золотой записи и создаем новый дата фрейм

In [29]:
# Создаем DataFrame из исходных данных
df = pd.DataFrame(games_df_start)

# Убираем лишние кавычки из названий столбцов
df.columns = df.columns.str.replace('"', '', regex=False)

# Функция для нахождения самой полной записи
def get_most_complete_record(group):
    # Считаем количество ненулевых значений в каждой строке
    completeness = group.notnull().sum(axis=1)

    # Находим индекс строки с максимальной полнотой
    best_index = completeness.idxmax()

    return group.loc[[best_index]]  # Возвращаем строку с максимальной полнотой

# Группируем по client_id и применяем функцию
gold_records = df.groupby(['client_id']).apply(get_most_complete_record).reset_index(drop=True)

# Заменяем NaN на "НЕТ ДАННЫХ"
gold_records.fillna("НЕТ ДАННЫХ", inplace=True)

# Устанавливаем параметры отображения для полного вывода
pd.set_option('display.max_rows', None)  # Отображаем все строки
pd.set_option('display.max_columns', None)  # Отображаем все столбцы

<ipython-input-29-e8d6dfe47a54>:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gold_records = df.groupby(['client_id']).apply(get_most_complete_record).reset_index(drop=True)


##### Вывод

In [30]:
display(gold_records)

,client_id,client_first_name,client_middle_name,client_last_name,client_fio_full,client_bday,client_bplace,client_cityzen,client_resident_cd,client_gender,client_marital_cd,client_graduate,client_child_cnt,client_mil_cd,client_zagran_cd,client_inn,client_snils,client_vip_cd,contact_vc,contact_tg,contact_other,contact_email,contact_phone,addr_region,addr_country,addr_zip,addr_street,addr_house,addr_body,addr_flat,addr_area,addr_loc,addr_city,addr_reg_dt,addr_str,fin_rating,fin_loan_limit,fin_loan_value,fin_loan_debt,fin_loan_percent,fin_loan_begin_dt,fin_loan_end_dt,stream_favorite_show,stream_duration,create_date,update_date,source_cd;;;;;;;;
0,,"""""""""",НЕТ ДАННЫХ,,,"""""""""","""""""""","""""""""","""""""""","""""""""","""""""""",navalihin@gothicgirl.com,+7 992 542 1065,55,1103.0,"""""632102""""",БОЛЬНИЧНАЯ,"""""11""""","""""""""","""""""""",КАЛАЧИНСКИЙ,"""""""""","""""""""",НЕТ ДАННЫХ,"""""55",КАЛАЧИНСКИЙ р-он,Г. КАЛАЧИНСК,ул. БОЛЬНИЧНАЯ,д.11,"""""",НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,2011-06-04 12:01:58.452,2011-06-04 12:01:58.452,"ISP"";;;;;;;",НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ
1,,"""""""""",НЕТ ДАННЫХ,,,"""""""""","""""""""","""""""""","""""""""","""""""""","""""""""",navalihin@gothicgirl.com,+7 992 542 1065,55.0,1103.0,"""""632102""""",БОЛЬНИЧНАЯ,"""""11""""","""""""""","""""""""",КАЛАЧИНСКИЙ,"""""""""","""""""""",НЕТ ДАННЫХ,"""""55",КАЛАЧИНСКИЙ р-он,Г. КАЛАЧИНСК,ул. БОЛЬНИЧНАЯ,д.11,"""""",НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,2019-03-08 18:03:35.064,2019-03-08 18:03:35.064,"Telco"";;;;;;;",НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ
2,П. ЗАУРАЛЬСКИЙ,ул. ТРАКТОРОСТРОИТЕЛЕЙ,д.14,"кв.2""""",7,1237800,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,2018-07-01 22:00:25.120,2018-07-01 22:00:25.120,"Bank2"";;;;;;",НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ
3,П. ЗАУРАЛЬСКИЙ,ул. ТРАКТОРОСТРОИТЕЛЕЙ,д.14,"кв.2""""",3.0,2460100.0,2274000.0,1137287.0,14.0,2018-02-06,2020-12-02,НЕТ ДАННЫХ,НЕТ ДАННЫХ,2018-03-07 04:49:07.428,2022-05-21 16:43:08.911,"Bank2"";;;;;;",НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ
4,П. ЗАУРАЛЬСКИЙ,ул. ТРАКТОРОСТРОИТЕЛЕЙ,д.14,"кв.2""""",3.0,2460100.0,2274000.0,1137287.0,14.0,2018-02-06,2020-12-02,НЕТ ДАННЫХ,НЕТ ДАННЫХ,2016-10-15 07:17:33.750,2024-09-10 15:11:20.350,"Bank2"";;;;;;",НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ
5,П. ЗАУРАЛЬСКИЙ,ул. ТРАКТОРОСТРОИТЕЛЕЙ,д.14,"кв.2""""",3.0,2460100.0,2274000.0,1137287.0,14.0,2018-02-06,2020-12-02,НЕТ ДАННЫХ,НЕТ ДАННЫХ,2017-10-15 13:24:14.448,2017-10-15 13:24:14.448,"Bank"";;;;;;",НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ ДАННЫХ
6,П. ЗАУРАЛЬСКИЙ,ул. ТРАКТОРОСТРОИТЕЛЕЙ,д.14,"кв.2""""",7.0,1237800.0,НЕТ ДАННЫХ,НЕТ ДАННЫХ,НЕТ 